In [1]:
import sqlite3
import requests
import re
import time
import random
import logging
from datetime import datetime
from watchdog import perekrestok_finder


logging.basicConfig(filename='operations.log', format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG)
db_address = '/home/alexey/Experiments/Fisherman_Bot/fisherman/fisherman_dev.db'

* add item
* get list of id:names
* remove items by id

### Add item

In [2]:
def add_item_to_database(url, user):
    """
    Add new item to watchlist. If not, add new entry to info table and username to users
    """
    
    # get info about item
    try:
        res = perekrestok_finder(url.strip())
        data = {
            'name': res[0],
            'url': url,
            'user': user
        }
    except Exception as err:
        logging.error("Database write new entry: can't obtain data for %s, %s", url, err)
        return f"There is an error during parsing {url}. Check it again or report to admins"
    

    # read data from DB
    con = sqlite3.connect(db_address)
    cur = con.cursor()
    logging.debug("Database write new entry: open connection")
    
    # check this url already in the database
    cur.execute('''SELECT id, url FROM info WHERE url = :url''', data)
    if not cur.fetchall():
        print("new entry")
        cur.execute('''INSERT INTO info (url, name) VALUES (:url, :name)''', data)
        logging.info(f"Database write new entry: {data['name']} was added to the database")
    cur.execute('''SELECT id FROM info WHERE url = :url''', data)
    data['ID'] = cur.fetchone()[0]
    
    # check, is this user already in the database
    cur.execute('''SELECT id, user FROM users WHERE user = :user AND id = :ID''', data)
    if not cur.fetchall():
        cur.execute('''INSERT INTO users (id, user) VALUES (:ID, :user)''', data)
        logging.info(f"Database write new entry: id {data['ID']} was added for {data['user']}")
        answer = "Item was added to you tracklist"
    else:
        answer = "Item is already in your watchlist"
    
    # write entries to DB
    con.commit()
    con.close()
    logging.debug("Database write new entry: close connection")
    
    return answer

* add non-existed item
* add existed item as new user
* add existed item as existed user
* add wrong item

In [4]:
add_item_to_database('https://www.perekrestok.ru/cat/367/p/arbuz-77410', 'root')

new entry


'Item was added to you tracklist'

In [5]:
add_item_to_database('https://www.perekrestok.ru/cat/367/p/arbuz-77410', 'abo')

'Item was added to you tracklist'

In [6]:
add_item_to_database('https://www.perekrestok.ru/cat/367/p/arbuz-77410', 'abo')

'Item is already in your watchlist'

In [7]:
add_item_to_database('https://www.perekrestok.ru/cat/367/p/arz-77892', 'abo')

'There is an error during parsing https://www.perekrestok.ru/cat/367/p/arz-77892. Check it again or report to admins'

---

### Get list of id:names

In [28]:
def get_list_of_items_on_watch(user):
    '''
    
    '''
    
    # read data from DB
    con = sqlite3.connect(db_address)
    cur = con.cursor()
    logging.debug("Database fetch data: open connection")
    
    # select id and names for current user
    cur.execute('''SELECT id, name FROM info WHERE id IN (SELECT id FROM users WHERE user = ?)''', [user])
    data = cur.fetchall()
    
    # close DB
    con.close()
    logging.debug("Database fetch data: close connection")
    
    if data:
        return data
    else:
        return "You do not have a list of items"

* known user
* unknown user

In [24]:
get_list_of_items_on_watch('abo')

(154, 'Арбуз')


In [30]:
get_list_of_items_on_watch('root3')

'You do not have a list of items'

---

### Remove item by ID

In [31]:
def remove_item_from_watch(ID, user):
    '''
    
    '''
    data = {
        "ID": ID,
        "user": user
    }
    
    
    # read data from DB
    con = sqlite3.connect(db_address)
    cur = con.cursor()
    logging.debug("Database remove data: open connection")
    
    # is this id in user watch list?
    cur.execute('''SELECT id, user FROM users WHERE id = :ID AND user = :user''', data)
    if not cur.fetchone():
        answer = f"You do not have this id in watchlist: {ID}"
    else:
        cur.execute('''DELETE FROM users WHERE id = :ID AND user = :user''', data)
        logging.info(f"Database remove data: removed id {ID} for user {user}")
    
        # is this id an orphan now?
        cur.execute('''SELECT id, user FROM users WHERE id = :ID''', data)
        if not cur.fetchone():
            cur.execute('''DELETE FROM info WHERE id = :ID''', data)
            logging.info(f"Database remove data: id {ID} COMPLETELY REMOVED for all users")
        answer = f"The {ID} id is removed from your watchlist"
    
    # close DB
    con.commit()
    con.close()
    logging.debug("Database remove data: close connection")

* remove item for one user
* remove item for second user
* remove item for non-user